1. Data Loading and Preprocessing

In [1]:
# Import necessary libraries
import zipfile
import os
import pandas as pd
import re
import ast

# Define paths for data
zip_path = './archive.zip'  # Update this path as needed
extraction_dir = './email_data/'

# Unzip dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

# Load CSV files into DataFrames
email_details_df = pd.read_csv(os.path.join(extraction_dir, 'CSV', 'email_thread_details.csv'))
email_summaries_df = pd.read_csv(os.path.join(extraction_dir, 'CSV', 'email_thread_summaries.csv'))

# Convert timestamps and clean 'to' column
email_details_df['timestamp'] = pd.to_datetime(email_details_df['timestamp'], errors='coerce')
email_details_df['to'] = email_details_df['to'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Clean up email body content
def clean_email_body(text):
    text = re.sub(r"(-{2,}|={2,}|Original Message|Forwarded by|From:.*\n)", "", text)
    text = re.sub(r"\n{2,}", "\n", text)  # Limit consecutive newlines
    text = re.sub(r"\s{2,}", " ", text)  # Remove excessive whitespace
    return text.strip()

email_details_df['cleaned_body'] = email_details_df['body'].apply(clean_email_body)

# Merge summaries with email details
email_merged_df = pd.merge(email_details_df, email_summaries_df, on='thread_id', how='left')
email_merged_df['search_text'] = email_merged_df['cleaned_body'] + " " + email_merged_df['summary'].fillna("")

# Display basic dataset information
print("Basic Information of Email Details Dataset:")
print(email_details_df.info())
print("\nBasic Information of Email Summaries Dataset:")
print(email_summaries_df.info())

# Check sample data
print("\nSample of Email Merged Data:")
print(email_merged_df[['thread_id', 'subject', 'timestamp', 'from', 'to', 'search_text']].head())


Basic Information of Email Details Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21684 entries, 0 to 21683
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   thread_id     21684 non-null  int64         
 1   subject       21684 non-null  object        
 2   timestamp     21684 non-null  datetime64[ns]
 3   from          21684 non-null  object        
 4   to            21684 non-null  object        
 5   body          21684 non-null  object        
 6   cleaned_body  21684 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.2+ MB
None

Basic Information of Email Summaries Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4167 entries, 0 to 4166
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   thread_id  4167 non-null   int64 
 1   summary    4167 non-null   object
dtypes: int64(1

2. Embedding Layer

In [2]:
from sentence_transformers import SentenceTransformer

# Initialize SentenceTransformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each email's search text
email_merged_df['embedding'] = email_merged_df['search_text'].apply(lambda x: model.encode(x).tolist())

# Display sample embeddings
print("\nSample Embeddings:")
print(email_merged_df[['search_text', 'embedding']].head())


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Sample Embeddings:
                                         search_text  \
0  Sent:=09Tuesday, January 29, 2002 1:23 PM\nTo:...   
1  Sent:=09Thursday, January 31, 2002 12:08 PM\nT...   
2  Note to Stephanie Panus....\nStephanie...pleas...   
3  Sent:=09Tuesday, February 05, 2002 4:59 PM\nTo...   
4  Sent:=09Tuesday, May 28, 2002 8:15 AM\nTo:=09L...   

                                           embedding  
0  [-0.01898478902876377, -0.024607710540294647, ...  
1  [-0.02736855112016201, 0.033215370029211044, -...  
2  [-0.05185914784669876, 0.011234914883971214, 0...  
3  [-0.028716154396533966, 0.024174343794584274, ...  
4  [-0.015747541561722755, 0.04201092571020126, 0...  


3.1 Search Layer
Semantic Search

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# Function for semantic search
def semantic_search(query, top_k=5):
    query_embedding = model.encode(query)
    similarities = cosine_similarity([query_embedding], email_merged_df['embedding'].tolist()).flatten()
    indices = similarities.argsort()[-top_k:][::-1]
    return email_merged_df.iloc[indices][['thread_id', 'subject', 'timestamp', 'from', 'to', 'search_text']]

# Test semantic search function
print("\nSample Semantic Search Results:")
print(semantic_search("termination meeting"))



Sample Semantic Search Results:
       thread_id                               subject           timestamp  \
19682       3784                                 Exxon 1999-05-26 01:34:00   
19683       3784                                 Exxon 1999-07-06 01:35:00   
16356       3139  FW: EPMI List Non terminated parties 2002-01-02 09:51:36   
7512        1430                     FW: CGLF contract 2001-11-16 12:14:58   
16357       3139  FW: EPMI List Non terminated parties 2002-01-02 10:15:35   

                           from  \
19682           Elizabeth Sager   
19683           Elizabeth Sager   
16356  Presto, Kevin M. KPRESTO   
7512     Germany, Chris CGERMAN   
16357  Presto, Kevin M. KPRESTO   

                                                      to  \
19682                                    [John Malowney]   
19683                                    [John Malowney]   
16356  [Bentley, Corry Cbentle, Choate, Jason Jchoate...   
7512                         [Hilliard, Marlene

3.2 Entity-Based Search

In [4]:
import spacy

# Load NLP model for NER
nlp = spacy.load("en_core_web_sm")

# Entity extraction function
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Apply entity extraction
email_merged_df['entities'] = email_merged_df['body'].apply(extract_entities)

# Function to search by entity
def search_by_entity(entity_name):
    results = email_merged_df[email_merged_df['entities'].apply(lambda ents: any(entity_name in ent for ent in ents))]
    return results[['thread_id', 'subject', 'timestamp', 'from', 'to', 'search_text']]

# Test entity search function
print("\nSample Entity-Based Search Results:")
print(search_by_entity("Jeffrey"))



Sample Entity-Based Search Results:
       thread_id                                            subject  \
25             5                         RE: long form confirm/MDEA   
26             5                         RE: long form confirm/MDEA   
27             5                         RE: long form confirm/MDEA   
28             5                         RE: long form confirm/MDEA   
29             5                         RE: long form confirm/MDEA   
...          ...                                                ...   
20410       3922  FW: Albuquerque Region Outage Report Through D...   
20411       3922  FW: Albuquerque Region Outage Report Through D...   
20434       3927                                      FW: Cogentrix   
20575       3956                                   FW: contact info   
21439       4121                                           Saturday   

                timestamp                         from  \
25    2001-04-29 03:46:00                 Edward Sac

3.3 Topic-Based Search

In [5]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Vectorize text data for LDA
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = vectorizer.fit_transform(email_merged_df['search_text'])

# Apply Latent Dirichlet Allocation for topic modeling
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(doc_term_matrix)

# Assign topics to each email
email_merged_df['topic'] = lda.transform(doc_term_matrix).argmax(axis=1)

# Function to filter emails by topic
def search_by_topic(topic_num):
    return email_merged_df[email_merged_df['topic'] == topic_num][['thread_id', 'subject', 'timestamp', 'search_text']]

# Test topic-based search function
print("\nSample Topic-Based Search Results:")
print(search_by_topic(1))



Sample Topic-Based Search Results:
       thread_id                     subject           timestamp  \
0              1  FW: Master Termination Log 2002-01-29 11:23:42   
1              1  FW: Master Termination Log 2002-01-31 12:50:00   
2              1  FW: Master Termination Log 2002-02-05 15:03:35   
3              1  FW: Master Termination Log 2002-02-05 15:06:25   
4              1  FW: Master Termination Log 2002-05-28 07:20:35   
...          ...                         ...                 ...   
21667       4164                presentation 2001-04-02 10:47:00   
21668       4164                presentation 2001-04-20 19:18:00   
21669       4164                presentation 2001-07-31 10:26:00   
21670       4164                presentation 2001-11-16 07:10:53   
21679       4166                    vacation 2000-10-04 11:32:00   

                                             search_text  
0      Sent:=09Tuesday, January 29, 2002 1:23 PM\nTo:...  
1      Sent:=09Thursday, Janu

4. Generation Layer

In [6]:
import torch
from transformers import pipeline

# Check if GPU is available
device = 0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU

# Initialize the summarizer with a smaller, faster model and GPU support if available
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=device)

def batch_summarize(texts, max_length=50, min_length=10):
    """
    Summarizes a batch of email bodies using GPU if available, with fixed max_length and min_length.
    """
    # Truncate each text in the batch to ensure it's within model input limits
    truncated_texts = [text[:512] if isinstance(text, str) and len(text) > 512 else text for text in texts]

    # Generate summaries in a batch
    try:
        summaries = summarizer(truncated_texts, max_length=max_length, min_length=min_length, do_sample=False)
        return [summary['summary_text'] for summary in summaries]
    except Exception as e:
        # Return an error message in case of an issue
        return [f"Error summarizing text: {e}" for _ in texts]

# Process the email bodies in batches and track progress
batch_size = 16  # You can adjust this batch size based on available GPU memory
summaries = []
total_emails = len(email_merged_df)

for i in range(0, total_emails, batch_size):
    # Print the current progress in console
    print(f"Processing emails {i + 1} to {min(i + batch_size, total_emails)} out of {total_emails}...")

    # Get the current batch of texts
    batch_texts = email_merged_df['body'].iloc[i:i+batch_size].tolist()
    batch_summaries = batch_summarize(batch_texts)
    summaries.extend(batch_summaries)

# Store the summaries in the DataFrame
email_merged_df['generated_summary'] = summaries

# Display sample summaries to confirm it works correctly
print("\nGenerated Summaries:")
print(email_merged_df[['thread_id', 'subject', 'generated_summary']].head())


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Processing emails 21649 to 21664 out of 21684...
Processing emails 21665 to 21680 out of 21684...


Your max_length is set to 50, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)
Your max_length is set to 50, but your input_length is only 28. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Your max_length is set to 50, but your input_length is only 47. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 50, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_

Processing emails 21681 to 21684 out of 21684...


Your max_length is set to 50, but your input_length is only 49. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)



Generated Summaries:
   thread_id                     subject  \
0          1  FW: Master Termination Log   
1          1  FW: Master Termination Log   
2          1  FW: Master Termination Log   
3          1  FW: Master Termination Log   
4          1  FW: Master Termination Log   

                                   generated_summary  
0   Messages sent January 29, 2002: =09Theriot, K...  
1   Panus, Stephanie Panus is 20-something-ish . ...  
2   Please remove my name as well as Melissa Murp...  
3   Panus: =09Panus, Stephanie Panus. =20.       ...  
4   CNG LDC (Hope Gas) termination 12/1 and the $...  


Testing

In [7]:
# Define the sample queries for testing
queries = [
    "termination meeting",                # Query 1
    "scheduled lunch meeting",            # Query 2
    "progress on friend's project"        # Query 3
]

# Function to test each query in the Search Layer and Generation Layer separately
def test_query_for_screenshots(query, top_k=3):
    print(f"\nTesting for Query: {query}")
    print("="*60)

    # Search Layer - Retrieve top 3 results
    print("\n**Top 3 Results from Search Layer (Semantic Search):**")
    semantic_results = semantic_search(query, top_k)
    print(semantic_results[['thread_id', 'subject', 'timestamp', 'from', 'to', 'search_text']])

    # Capture screenshot here for Search Layer

    # Generation Layer - Generate a summary based on the top results
    # Combine the top search results for summarization
    combined_text = " ".join(semantic_results['search_text'].tolist())
    generated_summary = batch_summarize([combined_text])[0]
    print("\n**Final Generated Answer from Generation Layer:**")
    print(generated_summary)

    # Capture screenshot here for Generation Layer
    print("="*60)

# Run tests for each query and capture screenshots
for query in queries:
    test_query_for_screenshots(query)



Testing for Query: termination meeting

**Top 3 Results from Search Layer (Semantic Search):**
       thread_id                               subject           timestamp  \
19682       3784                                 Exxon 1999-05-26 01:34:00   
19683       3784                                 Exxon 1999-07-06 01:35:00   
16356       3139  FW: EPMI List Non terminated parties 2002-01-02 09:51:36   

                           from  \
19682           Elizabeth Sager   
19683           Elizabeth Sager   
16356  Presto, Kevin M. KPRESTO   

                                                      to  \
19682                                    [John Malowney]   
19683                                    [John Malowney]   
16356  [Bentley, Corry Cbentle, Choate, Jason Jchoate...   

                                             search_text  
19682  John\nI'm not in on Friday so is there another...  
19683  John\nDennis and I spoke last week but I had t...  
16356  Sent:\tWednesday, January

Query Setup

In [9]:
# Define the sample queries for testing
queries = [
    "termination meeting",                # Query 1
    "scheduled lunch meeting",            # Query 2
    "progress on friend's project"        # Query 3
]


Output 1: Top 3 Results from the Search Layer for Query 1

In [10]:
# Output 1 - Top 3 Results from Search Layer for Query 1
print(f"\nTesting for Query: {queries[0]}")
print("="*60)

# Search Layer - Retrieve top 3 results for Query 1
print("\n**Top 3 Results from Search Layer (Semantic Search):**")
semantic_results_1 = semantic_search(queries[0], top_k=3)
print(semantic_results_1[['thread_id', 'subject', 'timestamp', 'from', 'to', 'search_text']])



Testing for Query: termination meeting

**Top 3 Results from Search Layer (Semantic Search):**
       thread_id                               subject           timestamp  \
19682       3784                                 Exxon 1999-05-26 01:34:00   
19683       3784                                 Exxon 1999-07-06 01:35:00   
16356       3139  FW: EPMI List Non terminated parties 2002-01-02 09:51:36   

                           from  \
19682           Elizabeth Sager   
19683           Elizabeth Sager   
16356  Presto, Kevin M. KPRESTO   

                                                      to  \
19682                                    [John Malowney]   
19683                                    [John Malowney]   
16356  [Bentley, Corry Cbentle, Choate, Jason Jchoate...   

                                             search_text  
19682  John\nI'm not in on Friday so is there another...  
19683  John\nDennis and I spoke last week but I had t...  
16356  Sent:\tWednesday, January

Output 2: Final Generated Answer from the Generation Layer for Query 1

In [11]:
# Output 2 - Final Generated Answer from Generation Layer for Query 1
combined_text_1 = " ".join(semantic_results_1['search_text'].tolist())
generated_summary_1 = batch_summarize([combined_text_1])[0]
print("\n**Final Generated Answer from Generation Layer:**")
print(generated_summary_1)



**Final Generated Answer from Generation Layer:**
 Dennis had promised to include Enron in any bid packages . But conflicts in schedules have caused delays . John suggests finding another day for the meeting and expresses concerns about preestablished term .


Output 3: Top 3 Results from the Search Layer for Query 2

In [12]:
# Output 3 - Top 3 Results from Search Layer for Query 2
print(f"\nTesting for Query: {queries[1]}")
print("="*60)

# Search Layer - Retrieve top 3 results for Query 2
print("\n**Top 3 Results from Search Layer (Semantic Search):**")
semantic_results_2 = semantic_search(queries[1], top_k=3)
print(semantic_results_2[['thread_id', 'subject', 'timestamp', 'from', 'to', 'search_text']])



Testing for Query: scheduled lunch meeting

**Top 3 Results from Search Layer (Semantic Search):**
       thread_id           subject           timestamp  \
13391       2561             lunch 2000-01-18 00:01:00   
16523       3169         FW: Lunch 2001-04-24 07:55:51   
7178        1370  RE: Lunch today? 2001-10-22 07:10:42   

                          from  \
13391        Richard B Sanders   
16523  Williams, Jason JWILLIA   
7178   Corman, Shelley SCORMAN   

                                                   to  \
13391                                  [Twanda Sweet]   
16523  [Williams, Jason EU/cn=Recipients/cn=JWilli13]   
7178                        [Holmes, Bradley Bholmes]   

                                             search_text  
13391  Please call him and set this up Richard B Sand...  
16523  Sent:\tTuesday, April 24, 2001 9:38 AM\nTo:\tL...  
7178   I now have another meeting over the lunch hour...  


Output 4: Final Generated Answer from the Generation Layer for Query 2

In [13]:
# Output 4 - Final Generated Answer from Generation Layer for Query 2
combined_text_2 = " ".join(semantic_results_2['search_text'].tolist())
generated_summary_2 = batch_summarize([combined_text_2])[0]
print("\n**Final Generated Answer from Generation Layer:**")
print(generated_summary_2)



**Final Generated Answer from Generation Layer:**
 Richard B Sanders suggests meeting at 12:00 but doesn't specify a location . nmanne@susmangodfrey.com suggests several dates for lunch .


Output 5: Top 3 Results from the Search Layer for Query 3

In [14]:
# Output 5 - Top 3 Results from Search Layer for Query 3
print(f"\nTesting for Query: {queries[2]}")
print("="*60)

# Search Layer - Retrieve top 3 results for Query 3
print("\n**Top 3 Results from Search Layer (Semantic Search):**")
semantic_results_3 = semantic_search(queries[2], top_k=3)
print(semantic_results_3[['thread_id', 'subject', 'timestamp', 'from', 'to', 'search_text']])



Testing for Query: progress on friend's project

**Top 3 Results from Search Layer (Semantic Search):**
      thread_id         subject           timestamp  \
5072        971  Monthly Update 2001-10-29 15:45:59   
5073        971  Monthly Update 2001-10-29 15:53:38   
1187        227          update 2000-06-20 02:56:00   

                             from                       to  \
5072  Gottfredson, Bryan BGOTTFRE      [Mara, Susan Smara]   
5073  Gottfredson, Bryan BGOTTFRE  [Kaufman, Paul Pkaufma]   
1187              Sara Shackleton            [Mark Taylor]   

                                            search_text  
5072  Sue - Would you please review the following re...  
5073  Paul - Please review the following report, and...  
1187  Mark: In connection with Susan F.'s departure,...  


Output 6: Final Generated Answer from the Generation Layer for Query 3

In [15]:
# Output 6 - Final Generated Answer from Generation Layer for Query 3
combined_text_3 = " ".join(semantic_results_3['search_text'].tolist())
generated_summary_3 = batch_summarize([combined_text_3])[0]
print("\n**Final Generated Answer from Generation Layer:**")
print(generated_summary_3)



**Final Generated Answer from Generation Layer:**
 Bryan is preparing a November report for Jim Steffes, and would like to have complete and accurate information about the projects you are involved in . Sue - Would you please review the following report, and advise me of any updates on projects
